# New-RscMssqlLiveMount
Rubrik offers a unique feature called Live Mount that does a full restore of a database to a specific point in time, but instead of copying the data back to the target SQL Server, Rubrik uses its own storage for the database. SQL Server will acces these files via a secured SMB3 share on the Rubrik cluster. 

New-RscMssqlLiveMount has multiple parameters that are needed to perform the Live Mount
- RscMssqlDatabase: This will be the object returned from Get-RscMssqlDatabase
- MountedDatabaseName: The name of the Live Mounted Database
- TargetMssqlInstance: This will be the object returned from Get-RscMssqlInstance
- Recovery Date Time: Thsi will be returned from Get-RscMssqlDatabaseRecoveryPoint    

# Workflow
To start an Live Mount of a database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the id of the host. This will be used in the next cmdlet when looking up a SQL Instance
1. Use **Get-RscMssqlInstance** to get the instance id of the Source SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscMssqlInstance** to get the instance id of the Target SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabaseRecoveryPoint** to get the fully formatted date that corresponds to the recovery point selected. 
1. Use **New-RscMssqlLiveMount** to start live mount of a database to a specific point in time. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC
 new-rscmssqllogBackup -RscMssqlDatabase $mssqldatabase  

## Connect to RSC

In [1]:
Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20240219-20     v20240219-20



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host ID

In [3]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Source Instance Information

In [4]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [5]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName -RscMssqlInstance $RscMssqlInstance

## Get the Target Instance Information

In [6]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows
$TargetMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the fully formatted recovery point

In [7]:
$RecoveryDateTime = Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -Latest

## Intiate an Live Mount of a database

In [9]:
$NewRscMssqlLiveMount = @{
    RscMssqlDatabase = $RscMssqlDatabase
    MountedDatabaseName = "test_livemount"
    TargetMssqlInstance = $TargetMssqlInstance
    RecoveryDateTime = $RecoveryDateTime
}
New-RscMssqlLiveMount @NewRscMssqlLiveMount


EndTime   : 
Id        : MSSQL_DB_MOUNT_e7bae1fc-e098-4575-9c0d-f2dc9aa9ad76_1476b6c4-8cc5-4240-a50d-97cfb10c11f
            f:::0
NodeId    : 
Progress  : 
StartTime : 
Status    : QUEUED
Error     : 
Links     : 



## Track the status of the Live Mount

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [10]:
Disconnect-Rsc

The Rubrik Security Cloud session has been terminated.
